# API for Pulling Bird Data from eBird

**Example data page:** American Coot; Cook County, IL; Mar-Sep 2010

https://ebird.org/barchart?byr=2010&eyr=2010&bmo=3&emo=9&r=US-IL-031&spp=treswa

Histogram data link: https://ebird.org/barchartData?r=US-IL-031&bmo=3&emo=9&byr=2010&eyr=2010&spp=treswa&fmt=tsv

Line data link: https://ebird.org/linegraph?bmo=3&emo=9&byr=2010&eyr=2010&r=US-IL-031&spp=treswa&fmt=tsv

In [50]:
import requests as r
from tqdm import tqdm_notebook

query = {
    'bmo':3, # beginning month
    'emo':9, # end month
    'byr':2010, # beginning year
    'eyr':2010, # end year
    'r':'US-IL-031', # location code for Cook County, IL
    'spp':'hirrus', # species 6-letter code from: https://www.birdpop.org/docs/misc/Alpha_codes_eng.pdf
    'fmt':'tsv', # data output format
}

bar_url = 'https://ebird.org/barchartData'
line_url = 'https://ebird.org/linegraph'

years = [2010,2011,2012,2013,2014,2015,2016,2017]

page = r.get(line_url, params=query)
data = page.text.split(sep='\n')

observations = {
    'species':data[4].split(sep='\t')[1],
    'week':data[2].split(sep='\t')[2:],
    'frequency_pct':pd.to_numeric(data[4].split(sep='\t')[2:]),
    'abundance':pd.to_numeric(data[8].split(sep='\t')[2:]),
    'birds_per_party_hr':pd.to_numeric(data[12].split(sep='\t')[2:]),
    'count_high':pd.to_numeric(data[16].split(sep='\t')[2:]),
    'count_total':pd.to_numeric(data[20].split(sep='\t')[2:]),
    'count_avg':pd.to_numeric(data[24].split(sep='\t')[2:]),
    'checklists_subs':pd.to_numeric(data[5].split(sep='\t')[2:]),
    'checklists_reports':pd.to_numeric(data[25].split(sep='\t')[2:]),
}

convert = pd.DataFrame(observations)
filename = 'ebird_{}-{}_{}_{}-{}.csv'.format(observations['species'], query['spp'], query['byr'], query['bmo'], query['emo'])
convert.to_csv(filename)
print('{} {} data saved as {}'.format(observations['species'], query['byr'], filename))

IndexError: list index out of range

## Create list of English bird names

In [69]:
with open('wnv_bird_list.txt') as f:
    data = f.readlines()

    collect = []
    
    for line in data:
        line = line.lower()
        line = line.replace(' exotic-captive', '\t')
        line = line.replace(' native-captive', '\t')
        line = line.replace(' native', '\t')
        line = line.replace(' introduced', '\t')
        line = line.replace('\t\n', '')
        line = line.replace('\t ', '\n')
        parts = line.split(sep='\n')
        for x in parts:
            collect.append(x)

with open('wnv_bird_list_clean.txt','w') as g:
    for x in sorted(collect):
        g.write('{}\n'.format(x))

## Convert list of bird names to 6-letter short names

In [72]:
with open('wnv_bird_list_clean.txt') as f:
    data = f.readlines()
    
    collect = []
    
    for line in data:
        if len(line.split(sep=' ')) == 0:
            new = line[:6]
        else:
            first = line.split(sep=' ')[0]
            last = line.split(sep=' ')[-1]
            new = first[:3] + last[:3]
        collect.append(new)

with open('wnv_bird_list_short.txt','w') as g:
    for x in sorted(collect):
        g.write('{}\n'.format(x)) 

## Scraping function

In [74]:
import requests as r
from tqdm import tqdm_notebook

query = {
    'bmo':3, # beginning month
    'emo':10, # end month
    'byr':2010, # beginning year
    'eyr':2010, # end year
    'r':'US-IL-031', # location code for Cook County, IL
    'spp':'hirrus', # species 6-letter code from: https://www.birdpop.org/docs/misc/Alpha_codes_eng.pdf
    'fmt':'tsv', # data output format
}

bar_url = 'https://ebird.org/barchartData'
line_url = 'https://ebird.org/linegraph'

years = [2010,2011,2012,2013,2014,2015,2016,2017]

def bird_scraper(yr_range, birds):
    results = {}
    for b in birds:
        results[b] = {}
    for y in tqdm_notebook(yr_range):
        query['byr'] = y;
        query['eyr'] = y;
        
        for x in birds:
            query['spp'] = x;
            
            try:
                page = r.get(line_url, params=query)
                data = page.text.split(sep='\n')


                observations = {
                    'species':data[4].split(sep='\t')[1],
                    'week':data[2].split(sep='\t')[2:],
                    'frequency_pct':pd.to_numeric(data[4].split(sep='\t')[2:]),
                    'abundance':pd.to_numeric(data[8].split(sep='\t')[2:]),
                    'birds_per_party_hr':pd.to_numeric(data[12].split(sep='\t')[2:]),
                    'count_high':pd.to_numeric(data[16].split(sep='\t')[2:]),
                    'count_total':pd.to_numeric(data[20].split(sep='\t')[2:]),
                    'count_avg':pd.to_numeric(data[24].split(sep='\t')[2:]),
                    'checklists_subs':pd.to_numeric(data[5].split(sep='\t')[2:]),
                    'checklists_reports':pd.to_numeric(data[25].split(sep='\t')[2:]),
                }

                convert = pd.DataFrame(observations)
                filename = 'ebird_{}-{}_{}_{}-{}.csv'.format(observations['species'], query['spp'], query['byr'], query['bmo'], query['emo'])
                convert.to_csv(filename)
                results[x][y] = filename
                print('{} {} data saved as {}'.format(observations['species'], query['byr'], filename))
            except:
                results[x][y] = 'ERROR'
    return results

In [78]:
f = open('wnv_bird_list_short.txt')
targets = f.readlines()
f.close()
cleaned = []
for x in targets:
    cleaned.append(x[:6])
cleaned

['abygro',
 'acowoo',
 'afrpar',
 'afrpen',
 'amecoo',
 'amecro',
 'amedip',
 'amegol',
 'amekes',
 'amepel',
 'amerob',
 'annhum',
 'ashfly',
 'baleag',
 'balori',
 'banpig',
 'banswa',
 'barowl',
 'barowl',
 'barswa',
 'belkin',
 'blachi',
 'blagro',
 'blaher',
 'blahum',
 'blalor',
 'blamag',
 'blapho',
 'blaski',
 'blaspa',
 'blavir',
 'blavul',
 'blawar',
 'blawar',
 'blawar',
 'blucon',
 'blujay',
 'blulor',
 'bluphe',
 'blytra',
 'boagra',
 'bobbob',
 'borowl',
 'brebla',
 'brocow',
 'brocow',
 'broduc',
 'brohaw',
 'brothr',
 'budbud',
 'bufbuf',
 'bulori',
 'burowl',
 'busbus',
 'cacgoo',
 'cacwre',
 'calcon',
 'calgul',
 'calqua',
 'caltow',
 'cancan',
 'cangoo',
 'canpar',
 'canwar',
 'carchi',
 'carwre',
 'casfin',
 'caster',
 'categr',
 'cedwax',
 'chechi',
 'chifla',
 'chigoo',
 'chirav',
 'chispa',
 'chiswi',
 'chuchu',
 'cintea',
 'cinvul',
 'clagre',
 'clanut',
 'clarai',
 'cliswa',
 'coccoc',
 'coccoc',
 'combla',
 'comcan',
 'comgol',
 'comgra',
 'comgro',
 'comloo',

In [80]:
bird_scraper(years, cleaned)

HBox(children=(IntProgress(value=0, max=8), HTML(value='')))

Acorn Woodpecker 2010 data saved as ebird_Acorn Woodpecker-acowoo_2010_3-10.csv
American Coot (Red-shielded) 2010 data saved as ebird_American Coot (Red-shielded)-amecoo_2010_3-10.csv
American Crow 2010 data saved as ebird_American Crow-amecro_2010_3-10.csv
American Dipper 2010 data saved as ebird_American Dipper-amedip_2010_3-10.csv
American Kestrel 2010 data saved as ebird_American Kestrel-amekes_2010_3-10.csv
American Robin 2010 data saved as ebird_American Robin-amerob_2010_3-10.csv
Anna's Hummingbird 2010 data saved as ebird_Anna's Hummingbird-annhum_2010_3-10.csv
Bald Eagle 2010 data saved as ebird_Bald Eagle-baleag_2010_3-10.csv
Baltimore Oriole 2010 data saved as ebird_Baltimore Oriole-balori_2010_3-10.csv
Bank Swallow 2010 data saved as ebird_Bank Swallow-banswa_2010_3-10.csv

Barn Swallow 2010 data saved as ebird_Barn Swallow-barswa_2010_3-10.csv
Blue Jay 2010 data saved as ebird_Blue Jay-blujay_2010_3-10.csv
Boreal Owl 2010 data saved as ebird_Boreal Owl-borowl_2010_3-10.csv

Northern Goshawk 2010 data saved as ebird_Northern Goshawk-norgos_2010_3-10.csv
Northern Mockingbird 2010 data saved as ebird_Northern Mockingbird-normoc_2010_3-10.csv
Northern Parula 2010 data saved as ebird_Northern Parula-norpar_2010_3-10.csv
Northern Pintail 2010 data saved as ebird_Northern Pintail-norpin_2010_3-10.csv
Northern Waterthrush 2010 data saved as ebird_Northern Waterthrush-norwat_2010_3-10.csv
Scaly-breasted Munia 2010 data saved as ebird_Scaly-breasted Munia-nutman_2010_3-10.csv
Nuttall's Woodpecker 2010 data saved as ebird_Nuttall's Woodpecker-nutwoo_2010_3-10.csv
Oak Titmouse 2010 data saved as ebird_Oak Titmouse-oaktit_2010_3-10.csv
Orchard Oriole 2010 data saved as ebird_Orchard Oriole-orcori_2010_3-10.csv
Pelagic Cormorant 2010 data saved as ebird_Pelagic Cormorant-pelcor_2010_3-10.csv
Peregrine Falcon 2010 data saved as ebird_Peregrine Falcon-perfal_2010_3-10.csv
Pinyon Jay 2010 data saved as ebird_Pinyon Jay-pinjay_2010_3-10.csv
Pine Siskin 2010 data saved as e

Common Merganser 2011 data saved as ebird_Common Merganser-commer_2011_3-10.csv
Common Murre 2011 data saved as ebird_Common Murre-commur_2011_3-10.csv
Common Nighthawk 2011 data saved as ebird_Common Nighthawk-comnig_2011_3-10.csv
Indian Peafowl 2011 data saved as ebird_Indian Peafowl-compea_2011_3-10.csv
Common Raven 2011 data saved as ebird_Common Raven-comrav_2011_3-10.csv
Common Yellowthroat 2011 data saved as ebird_Common Yellowthroat-comyel_2011_3-10.csv
Cooper's Hawk 2011 data saved as ebird_Cooper's Hawk-coohaw_2011_3-10.csv
Costa's Hummingbird 2011 data saved as ebird_Costa's Hummingbird-coshum_2011_3-10.csv
Downy Woodpecker 2011 data saved as ebird_Downy Woodpecker-dowwoo_2011_3-10.csv
Eastern Bluebird 2011 data saved as ebird_Eastern Bluebird-easblu_2011_3-10.csv
Eastern Kingbird 2011 data saved as ebird_Eastern Kingbird-easkin_2011_3-10.csv
Eastern Phoebe 2011 data saved as ebird_Eastern Phoebe-easpho_2011_3-10.csv
Eastern Towhee 2011 data saved as ebird_Eastern Towhee-eas

Iceland Gull (Thayer's) 2011 data saved as ebird_Iceland Gull (Thayer's)-thagul_2011_3-10.csv
Townsend's Warbler 2011 data saved as ebird_Townsend's Warbler-towwar_2011_3-10.csv
Tree Swallow 2011 data saved as ebird_Tree Swallow-treswa_2011_3-10.csv
Tricolored Blackbird 2011 data saved as ebird_Tricolored Blackbird-tribla_2011_3-10.csv
Tufted Titmouse 2011 data saved as ebird_Tufted Titmouse-tuftit_2011_3-10.csv
Tundra Swan 2011 data saved as ebird_Tundra Swan-tunswa_2011_3-10.csv
Turkey Vulture 2011 data saved as ebird_Turkey Vulture-turvul_2011_3-10.csv
Varied Thrush 2011 data saved as ebird_Varied Thrush-varthr_2011_3-10.csv
Virginia Rail 2011 data saved as ebird_Virginia Rail-virrai_2011_3-10.csv
Warbling Vireo 2011 data saved as ebird_Warbling Vireo-warvir_2011_3-10.csv
Western Bluebird 2011 data saved as ebird_Western Bluebird-wesblu_2011_3-10.csv
Western Kingbird 2011 data saved as ebird_Western Kingbird-weskin_2011_3-10.csv
Western Meadowlark 2011 data saved as ebird_Western Me

...

Mountain Chickadee 2015 data saved as ebird_Mountain Chickadee-mouchi_2015_3-10.csv
Mourning Dove 2015 data saved as ebird_Mourning Dove-moudov_2015_3-10.csv
Mountain Quail 2015 data saved as ebird_Mountain Quail-mouqua_2015_3-10.csv
Muscovy Duck 2015 data saved as ebird_Muscovy Duck-musduc_2015_3-10.csv
Mute Swan 2015 data saved as ebird_Mute Swan-mutswa_2015_3-10.csv
Nashville Warbler 2015 data saved as ebird_Nashville Warbler-naswar_2015_3-10.csv
Northern Bobwhite 2015 data saved as ebird_Northern Bobwhite-norbob_2015_3-10.csv
Northern Cardinal 2015 data saved as ebird_Northern Cardinal-norcar_2015_3-10.csv
Northern Flicker 2015 data saved as ebird_Northern Flicker-norfli_2015_3-10.csv
Northern Goshawk 2015 data saved as ebird_Northern Goshawk-norgos_2015_3-10.csv
Northern Mockingbird 2015 data saved as ebird_Northern Mockingbird-normoc_2015_3-10.csv
Northern Parula 2015 data saved as ebird_Northern Parula-norpar_2015_3-10.csv
Northern Pintail 2015 data saved as ebird_Northern Pinta

Chimney Swift 2016 data saved as ebird_Chimney Swift-chiswi_2016_3-10.csv
Cinnamon Teal 2016 data saved as ebird_Cinnamon Teal-cintea_2016_3-10.csv
Clark's Grebe 2016 data saved as ebird_Clark's Grebe-clagre_2016_3-10.csv
Clark's Nutcracker 2016 data saved as ebird_Clark's Nutcracker-clanut_2016_3-10.csv
Cliff Swallow 2016 data saved as ebird_Cliff Swallow-cliswa_2016_3-10.csv
Island Canary 2016 data saved as ebird_Island Canary-comcan_2016_3-10.csv
Common Goldeneye 2016 data saved as ebird_Common Goldeneye-comgol_2016_3-10.csv
Common Grackle 2016 data saved as ebird_Common Grackle-comgra_2016_3-10.csv
Common Loon 2016 data saved as ebird_Common Loon-comloo_2016_3-10.csv
Common Merganser 2016 data saved as ebird_Common Merganser-commer_2016_3-10.csv
Common Murre 2016 data saved as ebird_Common Murre-commur_2016_3-10.csv
Common Nighthawk 2016 data saved as ebird_Common Nighthawk-comnig_2016_3-10.csv
Indian Peafowl 2016 data saved as ebird_Indian Peafowl-compea_2016_3-10.csv
Common Raven

Snow Goose 2016 data saved as ebird_Snow Goose-snogoo_2016_3-10.csv
Song Sparrow 2016 data saved as ebird_Song Sparrow-sonspa_2016_3-10.csv
Spotted Owl 2016 data saved as ebird_Spotted Owl-spoowl_2016_3-10.csv
Spotted Towhee 2016 data saved as ebird_Spotted Towhee-spotow_2016_3-10.csv
Steller's Jay 2016 data saved as ebird_Steller's Jay-stejay_2016_3-10.csv
Swainson's Hawk 2016 data saved as ebird_Swainson's Hawk-swahaw_2016_3-10.csv
Swamp Sparrow 2016 data saved as ebird_Swamp Sparrow-swaspa_2016_3-10.csv
Swainson's Thrush 2016 data saved as ebird_Swainson's Thrush-swathr_2016_3-10.csv
Tennessee Warbler 2016 data saved as ebird_Tennessee Warbler-tenwar_2016_3-10.csv
Iceland Gull (Thayer's) 2016 data saved as ebird_Iceland Gull (Thayer's)-thagul_2016_3-10.csv
Townsend's Warbler 2016 data saved as ebird_Townsend's Warbler-towwar_2016_3-10.csv
Tree Swallow 2016 data saved as ebird_Tree Swallow-treswa_2016_3-10.csv
Tricolored Blackbird 2016 data saved as ebird_Tricolored Blackbird-tribla_

Herring Gull 2017 data saved as ebird_Herring Gull-hergul_2017_3-10.csv
Hermit Thrush 2017 data saved as ebird_Hermit Thrush-herthr_2017_3-10.csv
Hooded Merganser 2017 data saved as ebird_Hooded Merganser-hoomer_2017_3-10.csv
Hooded Oriole 2017 data saved as ebird_Hooded Oriole-hooori_2017_3-10.csv
Hooded Warbler 2017 data saved as ebird_Hooded Warbler-hoowar_2017_3-10.csv
House Finch 2017 data saved as ebird_House Finch-houfin_2017_3-10.csv
House Sparrow 2017 data saved as ebird_House Sparrow-houspa_2017_3-10.csv
House Wren 2017 data saved as ebird_House Wren-houwre_2017_3-10.csv
Inca Dove 2017 data saved as ebird_Inca Dove-incdov_2017_3-10.csv
Kentucky Warbler 2017 data saved as ebird_Kentucky Warbler-kenwar_2017_3-10.csv
Lark Sparrow 2017 data saved as ebird_Lark Sparrow-larspa_2017_3-10.csv
Laughing Gull 2017 data saved as ebird_Laughing Gull-laugul_2017_3-10.csv
Lazuli Bunting 2017 data saved as ebird_Lazuli Bunting-lazbun_2017_3-10.csv
Least Bittern 2017 data saved as ebird_Least

{'abygro': {2010: 'ERROR',
  2011: 'ERROR',
  2012: 'ERROR',
  2013: 'ERROR',
  2014: 'ERROR',
  2015: 'ERROR',
  2016: 'ERROR',
  2017: 'ERROR'},
 'acowoo': {2010: 'ebird_Acorn Woodpecker-acowoo_2010_3-10.csv',
  2011: 'ebird_Acorn Woodpecker-acowoo_2011_3-10.csv',
  2012: 'ebird_Acorn Woodpecker-acowoo_2012_3-10.csv',
  2013: 'ebird_Acorn Woodpecker-acowoo_2013_3-10.csv',
  2014: 'ebird_Acorn Woodpecker-acowoo_2014_3-10.csv',
  2015: 'ebird_Acorn Woodpecker-acowoo_2015_3-10.csv',
  2016: 'ebird_Acorn Woodpecker-acowoo_2016_3-10.csv',
  2017: 'ebird_Acorn Woodpecker-acowoo_2017_3-10.csv'},
 'afrpar': {2010: 'ERROR',
  2011: 'ERROR',
  2012: 'ERROR',
  2013: 'ERROR',
  2014: 'ERROR',
  2015: 'ERROR',
  2016: 'ERROR',
  2017: 'ERROR'},
 'afrpen': {2010: 'ERROR',
  2011: 'ERROR',
  2012: 'ERROR',
  2013: 'ERROR',
  2014: 'ERROR',
  2015: 'ERROR',
  2016: 'ERROR',
  2017: 'ERROR'},
 'amecoo': {2010: 'ebird_American Coot (Red-shielded)-amecoo_2010_3-10.csv',
  2011: 'ebird_American Coot (R

In [101]:
import numpy as np

def stitcher(bird_key):
    template = 'ebird_{}_201{}_3-10.csv'
    years = range(8)
    output = pd.DataFrame()
    
    for x in years:
        year = '201{}'.format(x)
        dtypes = {
            0: np.int32,
            1: np.float64, 
            2: np.float64,
            3: np.int32,
            4: np.int32,
            5: np.float64,
            6: np.int32,
            7: np.int32,
            8: np.float64,
            9: str,
            10: str,
        }
        bird = pd.read_csv(template.format(bird_key, x), prefix=year, index_col=[-1], usecols=[1,2,3,4,5,6,7,8], parse_dates=True, infer_datetime_format=True, dtype=dtypes)
        output = pd.concat([output,bird], axis=1, join='outer')
    
    return output       

In [102]:
american_crow = stitcher('American Crow-amecro')
american_crow.head()

,abundance,birds_per_party_hr,checklists_reports,checklists_subs,count_avg,count_high,count_total,abundance,birds_per_party_hr,checklists_reports,...,count_avg,count_high,count_total,abundance,birds_per_party_hr,checklists_reports,checklists_subs,count_avg,count_high,count_total
frequency_pct,,,,,,,,,,,,,,,,,,,,,
7.611549,0.196850,4.268710,29.0,381.0,2.689655,17.0,78.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7.964602,0.327434,2.280813,10.0,113.0,3.800000,8.0,38.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9.440559,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.181818,3.640237,23.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9.615385,0.315385,29.142209,23.0,260.0,3.565217,29.0,82.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10.769231,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.200000,2.966163,42.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
